## Prolog Runnable

The PrologRunnable class allows the generation of langchain runnables that use Prolog rules to generate answers.

Let's use the following Prolog rules in the file family.pl:

parent(john, bianca, mary).\
parent(john, bianca, michael).\
parent(peter, patricia, jennifer).\
partner(X, Y) :- parent(X, Y, _).

In [1]:
#!pip install langchain-prolog

from langchain_prolog import PrologConfig, PrologRunnable

TEST_SCRIPT = 'family.pl'


There are three diferent ways to use a PrologRunnable to query Prolog:

### 1) Using a full predicate string

In [2]:
config = PrologConfig(rules_file=TEST_SCRIPT)
prolog = PrologRunnable(prolog_config=config)

We can pass a string representing a single predicate query. The invoke method will return `True`, `False` or a list of dictionaries with all the solutions to the query:

In [3]:
prolog.invoke('partner(X, Y)')

[{'X': 'john', 'Y': 'bianca'},
 {'X': 'john', 'Y': 'bianca'},
 {'X': 'peter', 'Y': 'patricia'}]

### 2) Using a default predicate

In [4]:
config = PrologConfig(
            rules_file=TEST_SCRIPT,
            default_predicate='partner'
        )
prolog = PrologRunnable(prolog_config=config)

When using a default predicate, only the arguments for the predicate are passed to the Prolog runable, as a single string. Following Prolog conventions, uppercase identifiers are variables and lowercase identifiers are values (atoms or strings):

In [5]:
prolog.invoke('peter, X')

[{'X': 'patricia'}]

### 3) Using a dictionary and a query schema

In [6]:
schema = PrologRunnable.create_schema('partner', ['man', 'woman'])
config = PrologConfig(
            rules_file=TEST_SCRIPT,
            query_schema=schema,
        )
prolog = PrologRunnable(prolog_config=config)

If a schema is defined, we can pass a dictionary using the names of the parameters in the schema as the keys in the dictionary. The values can represent Prolog variables (uppercase first letter) or strings (lower case first letter). A `None` value is interpreted as a variable and replaced with the key capitalized:

In [7]:
prolog.invoke({"man": None, "woman": "bianca"})

[{'Man': 'john'}, {'Man': 'john'}]

You can also pass a Pydantic object generated with the schema to the invoke method. Uppercase values are treated as variables:

In [8]:
args = schema(man='X', woman='Y')
prolog.invoke(args)

[{'X': 'john', 'Y': 'bianca'},
 {'X': 'john', 'Y': 'bianca'},
 {'X': 'peter', 'Y': 'patricia'}]